In [2]:
!pip freeze | grep scikit-learn

scikit-learn==1.5.0


In [3]:
!python -V

Python 3.10.13


In [4]:
import pickle
import pandas as pd
# import uuid
import numpy as np


In [5]:
year = 2023
month = 3
# taxi_type is 'yellow' or 'green'
taxi_type = 'yellow'

# make the output directory if it doesn't exist
!mkdir -p output
# make the output directory for the taxi type if it doesn't exist
!mkdir -p output/{taxi_type}

input_file = f'https://d37ci6vzurychx.cloudfront.net/trip-data/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet'
output_file = f'output/{taxi_type}/_tripdata_{year}-{month:02d}.parquet'

In [6]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [7]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    # n = len(df)
    # ride_ids = [str(uuid.uuid4()) for _ in range(n)]
    # df['ride_id'] = ride_ids
    df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')


    return df

In [8]:
def apply_model(input_file, model, output_file):
    df = read_data(input_file)

    dicts = df[categorical].to_dict(orient='records')
    X_val = dv.transform(dicts)
    y_pred = model.predict(X_val)

    std_pred = np.std(y_pred)
    print(f"Standard deviation of predicted duration: {std_pred}")

    df_result = pd.DataFrame()

    df_result['ride_id'] = df['ride_id']
    # df_result['tpep_pickup_datetime'] = df['tpep_pickup_datetime']
    # df_result['PULocationID'] = df['PULocationID']
    # df_result['DOLocationID'] = df['DOLocationID']
    # df_result['actual_duration'] = df['duration']
    df_result['predicted_duration'] = y_pred
    # df_result['diff'] = df_result['predicted_duration'] - df_result['actual_duration']
    df_result.to_parquet(
        output_file,
        engine='pyarrow', 
        compression=None,
        index=False
    )


In [9]:
apply_model(input_file, model, output_file)

Standard deviation of predicted duration: 6.247488852238703
